<a href="https://colab.research.google.com/github/ParthibMudi/Titanic---Machine-Learning-from-Disaster/blob/main/Titanic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240803%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240803T185042Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dad4bf4ddb351df68712ad73fc723685c65e5a1ba0bf0f882fb4ba028aa1c5949f9f686c1344eda7aac044d94ab6f00ea6c39a0444579e7c89c78d343b2944a5cb9ffdf0e755d3b91479f6d0b9be445c65e95fb054f554cae05b3f222278de940a6e397355b841b6b430752e1abd4f877f1e8fc3a4f36bb8283b95cf2e550f263a8e68cfa6b3d24c56902927f9bfe7c662979ceb6398cfb5766783e467b1e50f546032ff5ad9887f6cd0670baa748601c1120bed105caff11fa3d2c596887b1fbc0fc7d0bb28faade0c34c77174c81c375cecee6988094f36b2ee73b1fe076bfd533400521e6a0e17ce6d9411e74d6445f2fef7e551cc44100a987d2eb8fc4819'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
test_output=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')


In [ ]:
train

In [ ]:
test

In [ ]:
test_output

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.scatter( train['Fare'],train['Survived'], s=train['Pclass'], alpha=0.5)

# # Optionally, add labels and title
# plt.xlabel('Column 1')
# plt.ylabel('Column 2')
# plt.title('Scatter Plot between Column 1 and Column 2 with sizes from Column 3')

# # Show the plot
# plt.show()

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter( train['Fare'],train['Survived'],train['Pclass'], c='r', marker='o')

# # Adding labels
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# plt.show()

In [ ]:
train=train.drop(['Name','PassengerId','Ticket','Cabin'],axis=1)
train

In [ ]:
train.info()

In [ ]:
mapping = {'C': 1, 'Q': 0, 'S': -1}
train['Embarked']=train['Embarked'].map(mapping)
train

In [ ]:
u=train['Sex'].unique()
print(u)

In [ ]:
mapping= {'male':1,'female':0}
train['Sex']=train['Sex'].map(mapping)
train

In [ ]:
train=train.dropna()
train

In [ ]:
test = train['Survived']
y_train =test

In [ ]:
train=train.drop('Survived',axis=1)
train

In [ ]:
x_test=pd.read_csv('/kaggle/input/titanic/test.csv')
y_test_output=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')


In [ ]:
x_test = x_test.drop(['PassengerId','Name','Cabin'],axis=1)
x_test


In [ ]:
x_test = x_test.drop(['Ticket'],axis=1)
x_test


In [ ]:
mapping= {'male':1,'female':0}
x_test['Sex']=x_test['Sex'].map(mapping)
x_test


In [ ]:
mapping = {'C': 1, 'Q': 0, 'S': -1}
x_test['Embarked']=x_test['Embarked'].map(mapping)
x_test

In [ ]:
y_test=y_test_output

In [ ]:
y_train =test
x_train=train
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)

In [ ]:
x_test_scaled = scaler.fit_transform(x_test)

In [ ]:
y_test=y_test.drop('PassengerId',axis=1)


In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(7,)),  # 7 input features
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(x_test_scaled, y_test))

Nureal network(ANN)

In [ ]:
# Example model using Sequential API
model = Sequential([
    Dense(32, activation='relu', input_shape=(7,)),  # Specify input_shape on the first layer
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)